In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.utils import class_weight
import numpy as np
import seaborn as sns
from tensorflow.keras.applications import DenseNet121

In [ ]:
# Configuración de rutas
base_dir = 'data'  # Reemplaza con la ruta principal donde están train, test, valid
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
valid_dir = os.path.join(base_dir, 'valid')

In [ ]:
# Generador de datos con normalización y aumento ajustado para el entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,  # Añadir flips verticales si es necesario
    brightness_range=[0.9, 1.1],  # Ajustar el rango de brillo
    fill_mode='nearest'
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

# Crear los generadores
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True  # Mezclar los datos en cada época
)

validation_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Cargar el modelo base (DenseNet121)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Descongelar las últimas capas del modelo base DenseNet121
base_model.trainable = True
for layer in base_model.layers[:-50]:  # Descongelar solo las últimas 50 capas
    layer.trainable = False

In [ ]:
# Añadir capas superiores personalizadas con regularización L2 y Dropout adicional
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(2, activation='softmax')(x)

# Crear el modelo completo
model = models.Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo con tasa de aprendizaje inicial ajustada
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy', AUC(name='auc')])

# Mostrar la arquitectura del modelo
model.summary()

# Configurar los pasos por época basados en el tamaño de datos y batch size
steps_per_epoch = int(np.ceil(train_generator.samples / train_generator.batch_size))
validation_steps = int(np.ceil(validation_generator.samples / validation_generator.batch_size))

# Calcular los pesos de las clases basado en el número de muestras
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Guardar el mejor modelo durante el entrenamiento
checkpoint_path = 'models/best_melanomaornot_model_from_scratch.keras'
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_auc',  # Monitorear AUC en lugar de val_loss
                             mode='max',
                             save_best_only=True,
                             verbose=1)

# Parar temprano si no hay mejora
early_stopping = EarlyStopping(monitor='val_auc', patience=8, verbose=1, restore_best_weights=True, mode='max')

# Reducir la tasa de aprendizaje si no hay mejora
reduce_lr = ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=4, verbose=1, min_lr=1e-7, mode='max')

In [ ]:
# Registrar el historial de entrenamiento para reanudarlo en caso de interrupción
csv_logger = CSVLogger('training_log_from_scratch.csv', append=True)

# Callback personalizado para calcular AUC-ROC en cada época
class RocAucCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_pred = self.model.predict(validation_generator)
        val_true = validation_generator.classes
        auc = roc_auc_score(val_true, val_pred[:, 1])
        print(f'Epoch {epoch + 1} - val_auc: {auc:.4f}')

In [ ]:
# Entrenar el modelo desde cero (sin cargar pesos previos)
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=30,
    callbacks=[checkpoint, early_stopping, reduce_lr, csv_logger, RocAucCallback()],
    class_weight=dict(enumerate(class_weights))  # Ajustar los pesos de las clases
)

In [54]:
# Evaluar el modelo en el conjunto de pruebas
test_loss, test_accuracy, test_auc = model.evaluate(
    test_generator, steps=test_generator.samples // test_generator.batch_size
)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Test AUC: {test_auc}')

# Ajustar el umbral
threshold = 0.3  # Umbral ajustado para minimizar los falsos negativos

# Calcular el número de steps exactos para cubrir todas las muestras
steps = int(np.ceil(test_generator.samples / test_generator.batch_size))

# Obtener predicciones de probabilidades (en lugar de clases)
predictions = model.predict(test_generator, steps=steps, verbose=1)

# Ajustar el umbral de clasificación: Si la probabilidad de Melanoma >= umbral, predice Melanoma (1)
predicted_classes = (predictions[:, 1] >= threshold).astype(int)

# Asegúrate de que no falten imágenes al final del proceso
true_classes = test_generator.classes
assert len(true_classes) == len(predicted_classes), "Mismatch in number of predictions and true labels"

In [ ]:
# Calcular AUC-ROC
auc = roc_auc_score(true_classes, predictions[:, 1])
print(f'AUC-ROC: {auc:.2f}')

# Generar el reporte de clasificación
class_labels = list(test_generator.class_indices.keys())
if len(true_classes) == len(predicted_classes):
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)
else:
    print("Las longitudes de true_classes y predicted_classes no coinciden. No se puede generar el reporte.")

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Guardar el modelo actualizado
model.save('models/melanoma_3_from_scratch.keras')